In [ ]:
import asyncio
from crawl4ai import AsyncWebCrawler
from crawl4ai.async_configs import BrowserConfig, CrawlerRunConfig

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import PydanticOutputParser
from typing import List, Dict, Any
from pydantic import BaseModel, Field

async def crawl_news(url):  # Ensures the function is asynchronous
    """
    웹 페이지의 내용을 크롤링하여 마크다운 형식으로 반환합니다.
    
    Args:
        url (str): 크롤링할 웹 페이지의 URL
        
    Returns:
        str: 크롤링된 내용의 마크다운 텍스트
    """
    browser_config = BrowserConfig()  # Default browser configuration
    run_config = CrawlerRunConfig(
        excluded_tags=['form', 'header', 'footer', 'nav'],
        keep_data_attributes=False,
        only_text=True,
        exclude_external_links=True,    
        exclude_social_media_links=True,
        exclude_external_images=True
    )
    async with AsyncWebCrawler(config=browser_config) as crawler:
        result = await crawler.arun(
            url=url,
            config=run_config
        )
        return result.markdown  # Return clean markdown content


class NewsArticle(BaseModel):
    content: str = Field(description="정제된 뉴스 기사 내용")
    topic: str = Field(description="뉴스 기사의 주제")
    keywords: List[str] = Field(description="기사에서 추출한 관련 뉴스기사 검색용 핵심 키워드 또는 문장")
    
    def to_dict(self) -> Dict[str, Any]:
        return {"content": self.content, "keywords": self.keywords}

def extract_news_content(content):
    """
    LLM을 사용하여 뉴스 내용에서 필요한 부분만 추출합니다.
    
    Args:
        content (str): 뉴스 내용 (마크다운 형식)
        llm (LLM, optional): 사용할 LLM 객체
        
    Returns:
        str: 필요한 부분만 추출된 뉴스 내용
    """

    llm = ChatOpenAI(
        model="gpt-4o-mini",
        temperature=0
    )
    
    # Pydantic 출력 파서 설정
    article_parser = PydanticOutputParser(pydantic_object=NewsArticle)
    
    # LLM에 보낼 프롬프트 템플릿
    prompt = ChatPromptTemplate.from_template(
        """다음은 웹 크롤링을 통해 얻은 뉴스 기사 내용입니다. 
        아래 두 가지 작업을 수행해주세요:
        
        1. 뉴스 기사의 실제 내용(제목, 본문, 날짜, 작성자 등)만 추출하고, 
           광고, 메뉴, 푸터, 사이드바 등의 불필요한 내용은 모두 제거해주세요.
        2. 관련 기사를 검색하기 위한 핵심 키워드를 5-10개 추출해주세요. 짧은 문장도 가능합니다. 
        
        원본 내용:
        {content}
        
        {format_instructions}
        """
    )
    
    # 체인 구성 - 형식 지침 포함
    chain = prompt.partial(format_instructions=article_parser.get_format_instructions()) | llm | article_parser
    
    # 체인 실행
    result = chain.invoke({"content": content})
    
    return result

async def process_news_from_url(url):
    """
    URL에서 뉴스를 크롤링하고 LLM으로 필요한 부분만 추출합니다.
    
    Args:
        url (str): 뉴스 기사 URL
        llm (LLM, optional): 사용할 LLM 객체
        
    Returns:
        str: 필요한 부분만 추출된 뉴스 내용
    """
    # 뉴스 크롤링
    content = await crawl_news(url)  # Uses await for the async crawl_news function
    
    # 내용 추출
    extracted_content = extract_news_content(content)
    
    return extracted_content

In [13]:
url = "https://www.hankyung.com/article/2025040632927"
# markdown = crawl_news(url)
# print("동기식 호출 결과:", markdown)

result = await process_news_from_url(url)
print("처리 결과:")
print("추출된 주제:", result.topic)
print("추출된 내용:", result.content)
print("추출된 키워드:", result.keywords)

처리 결과:



처리 결과:



AttributeError: 'coroutine' object has no attribute 'topic'

In [17]:
# 질문 목록을 위한 Pydantic 모델 정의
class QuestionList(BaseModel):
    questions: List[str] = Field(description="뉴스 기사에 대한 질문 목록")

# 출력 파서 설정
question_parser = PydanticOutputParser(pydantic_object=QuestionList)

# 프롬프트 템플릿 생성
question_prompt = ChatPromptTemplate.from_template(
    """다음은 뉴스 기사 내용입니다:

    {content}

    이 기사를 읽고 이해가 안되는 부분에 대한 질문을 여러 개 생성해주세요.
    질문은 명확하고 구체적이어야 하며, 기사의 내용을 바탕으로 해야 합니다.
    각 질문은 독립적이어야 합니다.

    {format_instructions}
    """
)

# LLM 설정
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0
)

# 체인 구성
question_chain = question_prompt.partial(format_instructions=question_parser.get_format_instructions()) | llm | question_parser

# 프롬프트 실행 (result.content는 이전 셀에서 정의됨)
question_response = question_chain.invoke({"content": result.content})

# 결과 출력
print("LLM이 생성한 질문 목록:")
if question_response and hasattr(question_response, 'questions'):
    for i, question in enumerate(question_response.questions):
        print(f"{i+1}. {question}")
else:
    print("질문을 생성하지 못했습니다.")


LLM이 생성한 질문 목록:
1. 3국 경제통상장관회의는 어떤 목적을 가지고 개최되었나요?
2. 브라이언 샤츠 상원의원이 언급한 '가장 충격적인 이미지'는 무엇을 의미하나요?
3. 트럼프 대통령의 관세 드라이브가 미국 경제에 미치는 영향은 무엇인가요?
4. 한·중·일 3국의 장관들이 협력하기로 합의한 내용은 구체적으로 무엇인가요?
5. 이 기사가 보도된 날짜는 언제이며, 어떤 사건을 다루고 있나요?
6. 샤츠 의원이 '도널드 트럼프가 세계를 뭉치게 하는 것'이라고 언급한 이유는 무엇인가요?
7. 3국 장관들이 악수한 장면이 국내 네티즌들 사이에서 화제가 된 이유는 무엇인가요?
8. 관세 전쟁이란 무엇이며, 이 기사에서 어떤 맥락에서 언급되고 있나요?
9. 한·중·일 자유무역협정(FTA)의 추진이 왜 중요한가요?
10. 이 기사의 주요 인물들은 누구이며, 그들의 역할은 무엇인가요?


In [18]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain_core.tools import Tool
from langchain.agents import create_react_agent, AgentExecutor
from langchain import hub
from langchain_community.tools.tavily_search import TavilySearchResults

load_dotenv()

def get_background_info_search(query: str) -> str:
    """Searches for background information related to a question."""
    search = TavilySearchResults()
    results = search.run(query)
    return results

class NewsQnAAgent:
    def __init__(self, model_name="gpt-4o-mini", temperature=0):
        self.llm = ChatOpenAI(
            temperature=temperature,
            model_name=model_name,
        )
        self.tools = [
            Tool(
                name="Search Background Information",
                func=get_background_info_search,
                description="Useful for finding background information, context, or explanations for concepts mentioned in the question",
            )
        ]
        react_prompt = hub.pull("hwchase17/react")
        self.agent = create_react_agent(llm=self.llm, tools=self.tools, prompt=react_prompt)
        self.agent_executor = AgentExecutor(
            agent=self.agent,
            tools=self.tools,
            verbose=True,
            handle_parsing_errors=True
        )

    def answer_question(self, news_content, question):
        """
        Answer a question about the news article using search for background/context.
        Args:
            news_content (str): The extracted news content
            question (str): A question generated from the news article
        Returns:
            str: Detailed answer with background/context, in Korean
        """
        context_prompt = """
        You are an expert news explainer. Given a news article and a question about it, your job is to answer the question in detail, using the search tool for any facts, background, or context you need.
        
        News Content:
        {news_content}
        
        Question:
        {question}
        
        Your task:
        1. Use the search tool to find relevant, specific information to answer the question.
        2. Make targeted search queries for any concepts, people, events, or terms you need to explain.
        3. For each search, follow the ReAct format: Thought, Action, Observation.
        4. Repeat the cycle as needed until you have enough information.
        5. When ready, give your final answer in Korean, with clear, factual, and detailed explanation (at least 2 paragraphs).
        
        IMPORTANT: Always use the search tool for factual information. Do not rely on your own knowledge.
        
        Final answer must be in Korean.
        """
        prompt_template = PromptTemplate(
            template=context_prompt,
            input_variables=["news_content", "question"]
        )
        formatted_prompt = prompt_template.format_prompt(
            news_content=news_content,
            question=question,
        )
        result = self.agent_executor.invoke(
            input={"input": formatted_prompt}
        )
        return result["output"]

# Example usage in notebook:
# 질문 목록(question_response.questions)과 뉴스 내용(result.content)을 사용
qna_agent = NewsQnAAgent()

for idx, question in enumerate(question_response.questions):
    print(f"\n=== 질문 {idx+1}: {question} ===")
    answer = qna_agent.answer_question(result.content, question)
    print(answer)

/Users/localgroup/Documents/workspace/Graphed_News/graphed_news/.venv/lib/python3.9/site-packages/langsmith/client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(



=== 질문 1: 3국 경제통상장관회의는 어떤 목적을 가지고 개최되었나요? ===


> Entering new AgentExecutor chain...
I need to find specific information about the purpose of the 3-country economic and trade ministers' meeting (한·중·일 경제통상장관회의) mentioned in the news article. This will help me provide a detailed answer to the question.

Action: Search Background Information
Action Input: "한·중·일 경제통상장관회의 목적"
[{'title': "'美관세전쟁' 속 한중일 경제통상장관회의… - 한국무역협회", 'url': 'https://www.kita.net/board/totalTradeNews/totalTradeNewsDetail.do;JSESSIONID_KITA=FE3E233E9165C4227AA3F1D51EB611BF.Hyper?no=90765&siteId=1', 'content': '한중일 3국 경제통상장관들이 5년여 만에 한자리에 모여 3국 간 협력 필요성에 공감하고 경제·통상 협력을 확대해 나가자는 데 뜻을 모았다.', 'score': 0.767429}, {'title': '한중일 통상장관 악수하자 美 의원 "충격적"…무엇에 놀랐나 - 뉴스1', 'url': 'https://www.news1.kr/diplomacy/defense-diplomacy/5744261', 'content': '이 회의에서 한중일 3국은 자유무역협정(FTA) 추진 등 경제·통상 협력을 논의했는데, 이번 회의가 2019년 이후 5년 만에 개최된 것이자 트럼프 대통령의', 'score': 0.70287734}, {'title': '\'美관세전쟁\' 속 한중일 경제통상장관회의…"경제통상 협력 확대"(종합)', 'url': 'https://

KeyboardInterrupt: 